# 打造客製化看盤軟體進擊版

Bokeh==2.1.0

## 從python小資族的sql提取資料

若DB為pickle檔，須將pd.read_sql那行修改為read_pickle，並要注意格式。

如果有上過韓老師python小資族，可將course11.ipynb上頭用read_sql取資料的程式改為下部份取資料，輸入鼓號和資料開始日期，產出的data丟入後面的繪圖程式，即可產生互動式圖表。

In [1]:
import pandas as pd
import sqlite3
import os
import json
from talib import abstract


# def get_price_data(stock_id,date):
#     # connect to sql
#     conn = sqlite3.connect(os.path.join('data', "data.db"))
#     # read data from sql
#     df = pd.read_sql(f"select stock_id,證券名稱, date, 開盤價, 收盤價, 最高價, 最低價, 成交股數 from price where stock_id='{stock_id}' and date > '{date}'", conn,
#         index_col=['date'])
#     # rename the columns of dataframe
#     df.index=df.index.astype(str).str.split(" ").str[0]
#     df.rename(columns={'收盤價':'close','證券名稱':'name', '開盤價':'open', '最高價':'high', '最低價':'low', '成交股數':'volume'}, inplace=True)
#     df['MA5']=df['close'].rolling(5).mean()
#     df['MA10']=df['close'].rolling(10).mean()
#     df['MA20']=df['close'].rolling(20).mean()
#     df['MA60']=df['close'].rolling(60).mean()
#     df['MA120']=df['close'].rolling(120).mean()
#     df['volume']=df['volume']/1000

    
#     RSI = pd.DataFrame(abstract.RSI(df, timeperiod=12),columns=['RSI_12'])
#     RSI['RSI_36']=abstract.RSI(df, timeperiod=36)
#     RSI=RSI.to_dict()
#     STOCH = abstract.STOCH(df).to_dict()
#     MACD=abstract.MACD(df).to_dict()
#     basic=df.iloc[-1,:2].to_dict()
#     df=df.drop(columns=['stock_id','name']).to_dict()
#     data={'basic':basic,'price_df':df,'RSI':RSI,'STOCH' :STOCH,'MACD':MACD }
    
#     return data



# data_demo=get_price_data("1101","2013-12-01")
# with open('/Users/benbilly3/bokeh_explore/demo_data/2330_tw_price.json', 'w') as outfile:
#     json.dump(data_demo, outfile)


ModuleNotFoundError: No module named 'talib'

## 產生json檔demo給沒買課程的人

In [20]:
# 跳過
# route=os.getcwd()+'/demo_data/2330_tw_price.json'
# data_demo=get_price_data("1101","2013-12-01")
# with open(route, 'w') as outfile:
#     json.dump(data_demo, outfile)



# Bokeh官方靜態範例

連結：
https://docs.bokeh.org/en/latest/docs/gallery/candlestick.html

有日期不連續問題

In [4]:
from math import pi
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.sampledata.stocks import MSFT

df = pd.DataFrame(MSFT)[:50]
df["date"] = pd.to_datetime(df["date"])

inc = df.close > df.open
dec = df.open > df.close
w = 12*60*60*1000 # half day in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, title = "MSFT Candlestick")
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

p.segment(df.date, df.high, df.date, df.low, color="black")
p.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

# output_file("candlestick.html", title="candlestick.py example")

output_notebook()
show(p)  # open a browser

Loading BokehJS ...

# 互動式技術圖表(擴展版)

1.figure圖紙設定，bokeh各種models應用。

https://docs.bokeh.org/en/latest/docs/reference/plotting.html

2.處理假日日期造成的資料不連續問題，x_range overwrite技巧

https://stackoverflow.com/questions/23585545/how-do-i-make-bokeh-omit-missing-dates-when-using-datetime-as-x-axis

3.hover互動資料顯示

https://docs.bokeh.org/en/latest/docs/user_guide/tools.html

4.legend物件控制，控制線圖隱藏，將legend移到圖表外讓版面清晰。

https://stackoverflow.com/questions/26254619/position-of-the-legend-in-a-bokeh-plot

https://docs.bokeh.org/en/latest/docs/user_guide/interaction/legends.html


5.位移、縮放、十字線、重置、存檔工具

https://docs.bokeh.org/en/latest/docs/user_guide/tools.html

6.second y_ranges繪製(使用雙軸)

https://docs.bokeh.org/en/latest/docs/user_guide/plotting.html#userguide-plotting-twin-axes

7.利用vbar和segment快速繪製k線,並將均線帶入。

https://docs.bokeh.org/en/latest/docs/gallery/candlestick.html

8.建立技術線子圖組

https://docs.bokeh.org/en/latest/docs/user_guide/layout.html#userguide-layout

In [2]:
import pandas as pd
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.models import SingleIntervalTicker, LinearAxis, ColumnDataSource, HoverTool
from bokeh.palettes import Spectral4, Dark2
from bokeh.models import LinearAxis, Range1d, Legend
from bokeh.layouts import column


def technical_chart(json_df):
    # get data
    stock_data = json_df
    df = pd.DataFrame(stock_data['price_df']).reset_index().rename(columns={'index': 'date'})
    df = df.reset_index()
    inc = df.close > df.open
    dec = df.open > df.close
    inc_data = df[inc]
    dec_data = df[dec]
    df_source = ColumnDataSource(df)
    inc_source = ColumnDataSource(inc_data)
    dec_source = ColumnDataSource(dec_data)

    # set hover
    hover = HoverTool(
        tooltips=[
            ("date", "@date"),
            ("close", "@open"),
            ("open", "@close"),
            ("high", "@high"),
            ("low", "@low"),
            ("volume", "@volume")
        ],
        formatters={"@date": "datetime"}
    )

    hover_rsi_kd = HoverTool(
        tooltips=[
            ("date", "@date"),
            ("RSI_12", "@RSI_12"),
            ("RSI_36", "@RSI_36"),
            ("slowk", "@slowk"),
            ("slowd", "@slowd"),

        ],
        formatters={"@date": "datetime"}

    )

    hover_macd = HoverTool(
        tooltips=[
            ("date", "@date"),
            ("macd", "@macd"),
            ("macdsignal", "@macdsignal"),
            ("macdhist", "@macdhist"),
        ],
        formatters={"@date": "datetime"}

    )

    # set figure data
    basic_data = stock_data['basic']
    title = basic_data['stock_id'] + ' ' + basic_data['name'] + ' ' + 'technical_chart'
    x_end = len(df)
    show_init_num = 120
    x_start = x_end - show_init_num
    interval_freq = show_init_num / 12
    y_start, y_end = df['close'].min() * 0.95,df['close'].max() * 1.05
    p1 = figure(plot_width=1000, title=title, plot_height=500, x_range=(x_start, x_end), y_range=(y_start, y_end),
                tools=[hover, "pan,zoom_in,zoom_out,crosshair,reset,save"], toolbar_location="above" ,y_axis_label="price")
    p2 = figure(plot_width=1000, title='RSI&KD', plot_height=250, x_range=(x_start, x_end),
                background_fill_color="#fafafa", tools=[hover_rsi_kd, "pan,zoom_in,zoom_out,crosshair,reset,save"],
                toolbar_location="above")
    p3 = figure(plot_width=1000, title='MACD', plot_height=250, x_range=(x_start, x_end),
                background_fill_color="#fafafa", tools=[hover_macd, "pan,zoom_in,zoom_out,crosshair,reset,save"],
                toolbar_location="above")

    for fig in [p1, p2, p3]:
        fig.title.text_font_size = '16pt'

        # map dataframe indices to date strings and use as label overrides
        fig.xaxis.major_label_overrides = {
            i: date.strftime('%Y/%m/%d') for i, date in enumerate(pd.to_datetime(df["date"]))
            # pd.date_range(start='3/1/2000', end='1/08/2018')
        }
        fig.xaxis.ticker = SingleIntervalTicker(interval=interval_freq)

    # set k bar chart
    # use the *indices* for x-axis coordinates, overrides will print better labels
    p1.segment('index', 'high', 'index', 'low', color="black", source=df_source)
    p1.vbar('index', 0.5, 'open', 'close', fill_color="#eb2409", line_color="black", source=inc_source)
    p1.vbar('index', 0.5, 'open', 'close', fill_color="#00995c", line_color="black", source=dec_source)

    # set ma line
    ma_legend_items = []
    for ma_name, color in zip(["MA5", "MA10", "MA20", "MA60", "MA120"], Dark2[5]):
        ma_df = df[['index', 'date', 'close', 'open', 'high', 'low', 'volume', ma_name]]
        source = ColumnDataSource(ma_df)
        ma_line = p1.line(x="index", y=ma_name, line_width=2, color=color, alpha=0.8,
                          muted_color=color, muted_alpha=0.2, source=source)
        ma_legend_items.append((ma_name, [ma_line]))

    # set ma legend
    legend = Legend(items=ma_legend_items, location=(0, 250))
    p1.add_layout(legend, 'left')

    # set twinx for volume
    y2_start = df['volume'].min() * 0.95
    y2_end = df['volume'].max() * 2
    p1.extra_y_ranges = {"vol": Range1d(y2_start, y2_end)}
    p1.vbar('index', 0.5, 'volume', y_range_name='vol', color='blue', alpha=0.2, source=df_source)
    p1.add_layout(LinearAxis(y_range_name="vol" ,axis_label="vol"), 'right')

    # RSI Chart
    rsi_df = pd.DataFrame(stock_data['RSI']).reset_index().rename(columns={'index': 'date'})
    kd_df = pd.DataFrame(stock_data['STOCH']).reset_index().rename(columns={'index': 'date'})
    rsi_kd_df = pd.concat([rsi_df, kd_df], axis=1)
    rsi_source = ColumnDataSource(rsi_kd_df)

    rsi_kd_legend_items = []
    for index_name, color in zip(["RSI_12", "RSI_36", "slowk", "slowd"], Spectral4):
        index_line = p2.line('index', index_name, line_width=3, color=color, alpha=0.8, muted_color=color,
                             muted_alpha=0.2, source=rsi_source)

        rsi_kd_legend_items.append((index_name, [index_line]))

    # set rsi_kd legend
    legend = Legend(items=rsi_kd_legend_items, location=(0, 50))
    p2.add_layout(legend, 'left')

    # MACD Chart
    macd = pd.DataFrame(stock_data['MACD']).reset_index().rename(columns={'index': 'date'})
    macd = macd.reset_index()
    macd_source = ColumnDataSource(macd)

    macd_legend_items = []
    for index_name, color in zip(["macd", "macdsignal", "macdhist"], Dark2[3]):
        if index_name == "macdhist":
            index_line = p3.vbar('index', 0.5, index_name, color=color, alpha=0.8, muted_color=color, muted_alpha=0.2,
                                 source=macd_source)
        else:
            index_line = p3.line('index', index_name, line_width=3, color=color, alpha=0.8, muted_color=color,
                                 muted_alpha=0.2, source=macd_source)

        macd_legend_items.append((index_name, [index_line]))

    # set macd legend
    legend = Legend(items=macd_legend_items, location=(0, 50))
    p3.add_layout(legend, 'left')

    # set legend mode
    for fig in [p1, p2, p3]:
        # set legend
        fig.legend.label_text_font_size = '8pt'
        # use hide or mute
        fig.legend.click_policy = "hide"
        #     fig.add_layout(legend, 'left')
    
    # use brower output
#     output_file("candlestick.html", title="candlestick.py example")
    # use jupyter output
    output_notebook()
    # use columns to control all fig  locations,you could try row method.
    show(column(p1, p2, p3))


## 程式測試

In [7]:
#open 2330 TSMC 台積電 demo_file
route=os.getcwd()+'/demo_data/2330_tw_price.json'
with open(route, 'r') as outfile:
    test=json.load(outfile)

# run code
technical_chart(test)

{'basic': {'stock_id': '2330', 'name': '台積電'}, 'price_df': {'open': {'2013-12-02': 105.0, '2013-12-03': 104.0, '2013-12-04': 103.5, '2013-12-05': 105.0, '2013-12-06': 103.0, '2013-12-09': 104.0, '2013-12-10': 104.5, '2013-12-11': 103.5, '2013-12-12': 104.5, '2013-12-13': 102.5, '2013-12-16': 102.5, '2013-12-17': 103.0, '2013-12-18': 102.5, '2013-12-19': 102.0, '2013-12-20': 101.5, '2013-12-23': 103.0, '2013-12-24': 104.0, '2013-12-25': 102.5, '2013-12-26': 103.0, '2013-12-27': 103.5, '2013-12-30': 104.0, '2013-12-31': 105.5, '2014-01-02': 105.0, '2014-01-03': 103.0, '2014-01-06': 102.0, '2014-01-07': 102.5, '2014-01-08': 103.0, '2014-01-09': 102.5, '2014-01-10': 102.0, '2014-01-13': 103.0, '2014-01-14': 102.5, '2014-01-15': 104.0, '2014-01-16': 106.0, '2014-01-17': 108.0, '2014-01-20': 107.0, '2014-01-21': 108.5, '2014-01-22': 106.0, '2014-01-23': 108.5, '2014-01-24': 107.0, '2014-01-27': 106.0, '2014-02-05': 102.0, '2014-02-06': 101.0, '2014-02-07': 103.5, '2014-02-10': 104.0, '2014-0

Loading BokehJS ...

In [29]:
json_df=test
stock_data = json_df
df = pd.DataFrame(stock_data['price_df']).reset_index().rename(columns={'index': 'date'})
df = df.reset_index()

for i, date in enumerate(pd.to_datetime(df["date"])):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27